In [1]:
!pip install 'ibis-framework[trino]'


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [48]:
import ibis
import yaml
import trino
import pandas as pd
from trino.auth import OAuth2Authentication
import pystarburst
from pystarburst import Session
import pyarrow.parquet as pq
import datetime
from dateutil import parser
from datetime import datetime
import ibis.expr.types as ir
from ibis import _


ibis.options.interactive = True

In [49]:
with open ("galaxy_connection.yaml", "r") as creds:
    creds = yaml.safe_load(creds)

In [50]:
user = "yusuf.cattaneo@starburstdata.com/accountadmin"
password = creds['trino_password']
host = 'ycat-aws-demo.trino.galaxy.starburst.io'
port = '443'
catalog = "demo_aws_s3"
schema = "burst_bank"

con = ibis.trino.connect(
    user=user, password=password, host=host, port=port, 
    database=catalog, schema=schema, 
    auth=OAuth2Authentication(), http_scheme='https'
)
con.list_tables()

Open the following URL in browser for the external authentication:
https://ycat-aws-demo.trino.galaxy.starburst.io/oauth/v2/token/initiate/ce72bb5ba59c3eccddeea4a44fb64646103a5239f02c437a3965258c402588ff


['account',
 'accounts_table',
 'athena_view4',
 'auto_loan_payment',
 'credit_card_payment',
 'credit_card_payment_bulk',
 'customer',
 'customer_1',
 'customer_profile',
 'dl_customer_consume',
 'employee',
 'galaxy_lineage_test',
 'iceberg_cdc',
 'iceberg_customer_consume',
 'iceberg_customer_table',
 'iceberg_customer_table__dbt_tmp',
 'leverage_udf_like_macro',
 'mortgage_payment',
 'postgres_customer',
 'postgres_customer_1',
 'product_profile',
 'product_profile_ins',
 'state_census',
 'stg_snapshot_customer',
 'test_merge',
 'test_sb_view',
 'yc_test']

In [51]:
customer = con.table('customer', database='demo_aws_s3')
account = con.table('account', database='demo_aws_s3')

In [52]:
joined_table = customer.inner_join(account, customer.custkey == account.custkey)


In [53]:
# Select columns
join = joined_table[
    "acctkey",
    "registration_date",
    "cc_open_date"
]

# Execute the query
# result_df = result.execute()
join

┏━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ acctkey ┃ registration_date ┃ cc_open_date ┃
┡━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│ string  │ string            │ string       │
├─────────┼───────────────────┼──────────────┤
│ 1217470 │ 1966-09-08        │ 2000-06-30   │
│ 1217485 │ 1965-04-08        │ 1994-02-07   │
│ 1217486 │ 2017-12-21        │ 2008-07-18   │
│ 1217487 │ 1991-02-19        │ 2008-05-24   │
│ 1217488 │ 1951-04-24        │ 2019-01-18   │
│ 1217489 │ 2016-08-22        │ 2010-10-14   │
│ 1217490 │ 1941-10-26        │ 2008-09-11   │
│ 1217491 │ 1982-08-13        │ 2012-09-24   │
│ 1217492 │ 1933-01-11        │ 2003-08-31   │
│ 1217493 │ 2012-01-09        │ 2008-11-26   │
│ …       │ …                 │ …            │
└─────────┴───────────────────┴──────────────┘

In [54]:
diff_types = join.mutate(date_registration=join.registration_date.cast("date"))
diff_types

┏━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ acctkey ┃ registration_date ┃ cc_open_date ┃ date_registration ┃
┡━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ string  │ string            │ string       │ date              │
├─────────┼───────────────────┼──────────────┼───────────────────┤
│ 1217478 │ 1992-12-22        │ 2000-12-19   │ 1992-12-22        │
│ 1217479 │ 1962-04-06        │ 2014-08-14   │ 1962-04-06        │
│ 1217484 │ 1933-03-26        │ 2007-04-13   │ 1933-03-26        │
│ 1217596 │ 2017-05-13        │ 2016-06-06   │ 2017-05-13        │
│ 1217597 │ 1973-07-12        │ 1993-11-27   │ 1973-07-12        │
│ 1217598 │ 1943-07-25        │ 2015-04-13   │ 1943-07-25        │
│ 1217477 │ 1952-10-31        │ 2009-09-05   │ 1952-10-31        │
│ 1217480 │ 1997-03-14        │ 1994-05-26   │ 1997-03-14        │
│ 1217481 │ 1978-01-22        │ 1995-01-01   │ 1978-01-22        │
│ 1217482 │ 1934-08-16        │ 2018-01-29   │ 1934-08-16        │
│ …       │ …                 │ …            │ …                 │
└─────────┴───────────────────┴──────────────┴───────────────────┘

In [58]:
def calc_date_diff(start_date, end_date):
    return start_date.cast("date").delta(end_date.cast("date"), "day")
    

In [61]:
# date_test = diff_types.mutate(
#                             string_date_diff=calculate_date_diff(diff_types.registration_date, diff_types.cc_open_date)
#                             ) 
date_test = diff_types.mutate(string_date_diff=calc_date_diff(diff_types.date_registration, diff_types.cc_open_date)
                            )
date_calc = date_test.execute()
date_calc


,acctkey,registration_date,cc_open_date,date_registration,string_date_diff
0,1217473,2002-12-03,1993-01-10,2002-12-03,3614
1,1217474,1962-04-10,2012-10-09,1962-04-10,-18445
2,1217475,1976-04-20,1996-09-20,1976-04-20,-7458
3,1217476,1998-07-06,2004-04-20,1998-07-06,-2115
4,1217532,1953-12-02,2009-01-06,1953-12-02,-20124
...,...,...,...,...,...
995,1218465,2016-05-30,2007-01-16,2016-05-30,3422
996,1218466,1997-05-04,2004-12-15,1997-05-04,-2782
997,1218467,1985-05-22,2017-08-01,1985-05-22,-11759
998,1218468,2008-10-20,1997-10-22,2008-10-20,4016
